In [2]:
import gradio as gr
import os
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from openai import OpenAI



def run_eda_safe(file_obj):

    df = pd.read_csv(file_obj.name)
    output = "output"
    os.makedirs(output, exist_ok=True)

    # WRITE SUMMARY
    summary_path = os.path.join(output, "eda_summary.txt")
    with open(summary_path, "w") as f:
        f.write("Shape:\n")
        f.write(str(df.shape) + "\n\n")

        f.write("Dtypes:\n")
        f.write(str(df.dtypes) + "\n\n")

        f.write("Columns:\n")
        f.write(str(list(df.columns)) + "\n\n")

        f.write("Describe:\n")
        f.write(str(df.describe(include='all')) + "\n\n")

        f.write("Missing:\n")
        f.write(str(df.isnull().sum()) + "\n")

    images = []

    # --- Missing Plot ---
    missing = df.isnull().mean()
    if missing.sum() > 0:
        plt.figure(figsize=(8,4))
        sns.barplot(x=missing.index, y=missing.values)
        plt.xticks(rotation=45)
        p = os.path.join(output, "missing.png")
        plt.tight_layout()
        plt.savefig(p)
        plt.close()
        images.append(p)

    # --- Numeric Dist ---
    num_cols = df.select_dtypes(include="number").columns
    for col in num_cols:
        plt.figure(figsize=(6,3))
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(col)
        p = os.path.join(output, f"{col}_dist.png")
        plt.tight_layout()
        plt.savefig(p)
        plt.close()
        images.append(p)

    # --- Categorical ---
    cat_cols = df.select_dtypes(exclude="number").columns
    for col in cat_cols[:5]:
        plt.figure(figsize=(6,3))
        df[col].value_counts().head(10).plot(kind="bar")
        plt.title(col)
        plt.xticks(rotation=45)
        p = os.path.join(output, f"{col}_cat.png")
        plt.tight_layout()
        plt.savefig(p)
        plt.close()
        images.append(p)

    # --- Correlation ---
    if len(num_cols) > 1:
        corr = df[num_cols].corr().replace([np.inf, -np.inf], np.nan).fillna(0)
        plt.figure(figsize=(7,5))
        sns.heatmap(corr, annot=False, cmap="coolwarm")
        p = os.path.join(output, "corr.png")
        plt.tight_layout()
        plt.savefig(p)
        plt.close()
        images.append(p)

    return summary_path, images



def llm_narrative(summary_path):

    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key="API_KEY"
    )

    with open(summary_path, "r") as f:
        eda_summary = f.read()

    prompt = f"""
You are a Kaggle competition expert.
Provide:
1. Dataset description
2. Key trends
3. Preprocessing suggestions
4. Feature engineering ideas
5. Top 3 models

EDA Summary:
{eda_summary}
"""

    try:
        out = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        return out.choices[0].message.content

    except Exception as e:
        return f" LLM Error: {e}"



def pipeline(csv_file):
    summary_file, images = run_eda_safe(csv_file)
    narrative = llm_narrative(summary_file)
    return narrative, summary_file, images


with gr.Blocks() as demo:

    gr.Markdown("## 📊 Auto EDA + LLM Narrative (Colab Version)")

    file_input = gr.File(label="Upload CSV")
    run_btn = gr.Button("Run")

    out_narrative = gr.Markdown(label="Narrative Output")
    out_summary = gr.File(label="EDA Summary File")
    out_imgs = gr.Gallery(label="Plots", columns=3)

    run_btn.click(
        fn=pipeline,
        inputs=file_input,
        outputs=[out_narrative, out_summary, out_imgs]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b02f8ffce1daec3913.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
